In [3]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr connect --cluster-id j-3GWESRXJMF9D8 --auth-type None --language python  

Successfully read emr cluster(j-3GWESRXJMF9D8) details
Initiating EMR connection..
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1662658496356_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
{"namespace": "sagemaker-analytics", "cluster_id": "j-3GWESRXJMF9D8", "error_message": null, "success": true, "service": "emr", "operation": "connect"}


# Installing Python Packages to Existing EMR Clusters

## Notebook Scope Libraries
Notebook-scoped libraries provide you the following benefits:

* Runtime installation – You can import your favorite Python libraries from PyPI repositories and install them on your remote cluster (driver and executors) on the fly when you need them. These libraries are instantly available to your Spark runtime environment. There is no need to restart the notebook session or recreate your cluster.
* Dependency isolation – The libraries you install using EMR Notebooks are isolated to your notebook session and don’t interfere with bootstrapped cluster libraries or libraries installed from other notebook sessions. These notebook-scoped libraries take precedence over bootstrapped libraries. Multiple notebook users can import their preferred version of the library and use it without dependency clashes on the same cluster.
* Portable library environment – The library package installation happens from your notebook file. This allows you to recreate the library environment when you switch the notebook to a different cluster by re-executing the notebook code. At the end of the notebook session, the libraries you install through EMR Notebooks are automatically removed from the hosting EMR cluster.

#### This functionality is available for clusters running EMR release >= 5.26.0. More info can be found in the [docs here](https://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-managed-notebooks-installing-libraries-and-kernels.html)

In [4]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1662658496356_0003,pyspark,idle,Link,Link,None,
2,application_1662658496356_0004,pyspark,idle,Link,Link,None,✔


## To enable notebook scoped libraries, we must set the configuration to use a virtualenv

In [5]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1662658496356_0005,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1662658496356_0003,pyspark,idle,Link,Link,None,
3,application_1662658496356_0005,pyspark,idle,Link,Link,None,✔


In [6]:
sc.list_packages()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- ----------
aws-cfn-bootstrap          2.0
beautifulsoup4             4.9.3
boto                       2.49.0
boto3                      1.18.63
botocore                   1.21.65
certifi                    2022.6.15
charset-normalizer         2.0.12
click                      8.1.1
cycler                     0.11.0
Cython                     0.29.24
distlib                    0.3.6
docutils                   0.14
filelock                   3.8.0
idna                       3.3
importlib-metadata         4.12.0
jmespath                   0.10.0
joblib                     1.1.0
kiwisolver                 1.4.4
lockfile                   0.11.0
lxml                       4.8.0
matplotlib                 3.4.3
mysqlclient                1.4.2
nltk                       3.7
nose                       1.3.4
numpy                      1.21.2
pandas                     1.2.5
Pillow                     9.2.0
pip                       

## My work starts here

In [16]:
%%configure -f 

{
  "conf": {
    "spark.pyspark.python": "python3",
    "spark.pyspark.virtualenv.enabled": "true",
    "spark.pyspark.virtualenv.type":"native",
    "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
    "spark.jars.packages": "io.delta:delta-core_2.12:2.0.0",
    "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
    "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog"
  }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1662658496356_0008,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1662658496356_0003,pyspark,idle,Link,Link,None,
6,application_1662658496356_0008,pyspark,idle,Link,Link,None,✔


In [24]:
# from delta.tables import *

data = spark.read.csv("s3://vjain-dbx-bucket/nike-gt-poc/nike_product_full.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


In [ ]:
data.head()

## OG: Notice that there is no "translate" package available on our cluster

In [27]:
# Install translate from given PyPI repository
sc.install_pypi_package("translate", "https://pypi.org/simple") 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
from translate import Translator
from pyspark.sql.functions import udf, col

@udf
def translate_to_german(sentence):
    translator= Translator(to_lang="de")
    return translator.translate(sentence)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
columns = ["TaskID","Sentence"]
data = [("1", "This is a pen"),
    ("2", "This is a chair")]

df = spark.createDataFrame(data=data,schema=columns)

df.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df.select(col("TaskID"), \
    translate_to_german(col("Sentence")).alias("Sentence") ) \
   .show(truncate=False)